# Technique for Order of Preference by Similarity to Ideal Solution (AHP-TOPSIS)

##### Modified from: https://www.kaggle.com/code/hungrybluedev/topsis-implementation/notebook </br>

It is a multi-criteria decision analysis method that is based on the concept that the chosen alternative should have the shortest geometric distance to the Positive Ideal Solution (PIS) and the longest geometric solution from the Negative Ideal Solution (NIS).

# AHP Weight calculation
1.  Pair-wise comparison of each criteria and sub-criteria to establish the weight of the supply chain parameters.
2. Global summation of all these weights (weighted arithmetic sum) for each alternative and ordering them on the basis of this weighted sum.
3. Calculate the consistency ratio which should be less than 0.10, otherwise the weights are not balanced

## AHP Pair-wise Comparison Matrix
As the first step a pairwise comparison matrix is determined qualitatively according to the priority using the Saaty 9 point scale (Evelyn, E. et.al, 2015). This table is subjectively evaluated, as AHP combines both quantitative and qualitative aspects in the decision method, which helps the analysis to find the best possible answer rather than a correct solution (Longaray, A.A. et.al, 2015).
<br /><br />

| Features              | Digital Prepardness  | Natural Disaster | Labor Strikes | Political Stability | Logistic Index |
|-----------------------|----------------------|------------------|---------------|---------------------|----------------|
| Digital Prepardness   | 1                    | 6                | 8             | 4                   | 3              |
| Natural Disaster      | 1/6                  | 1                | 3             | 1/2                 | 1/4            |
| Labor Strikes         | 1/8                  | 1/3              | 1             | 1/2                 | 1/4            |
| Political Instability | 1/4                  | 2                | 2             | 1                   | 1/3            |
| Logistic Index        | 1/3                  | 5                | 4             | 3                   | 1              |

In [38]:
from ahpy import ahpy

supply_chain_comp = {
    ('Digital Prepardness', 'Digital Prepardness'): 1, ('Digital Prepardness', 'Natural Disasters'): 6, ('Digital Prepardness', 'Labor Strikes'): 8, ('Digital Prepardness', 'Political Stability'): 4, ('Digital Prepardness', 'Logistic Index'): 3,
    ('Natural Disasters', 'Digital Prepardness'): 1/6, ('Natural Disasters', 'Natural Disasters'): 1, ('Natural Disasters', 'Labor Strikes'): 3, ('Natural Disasters', 'Political Stability'): 1/2, ('Natural Disasters', 'Logistic Index'): 1/5,
    ('Labor Strikes', 'Digital Prepardness'): 1/8, ('Labor Strikes', 'Natural Disasters'): 1/3, ('Labor Strikes', 'Labor Strikes'): 1, ('Labor Strikes', 'Political Stability'): 1/2, ('Labor Strikes', 'Logistic Index'): 1/4,
    ('Political Stability', 'Digital Prepardness'): 1/4, ('Political Stability', 'Natural Disasters'): 2, ('Political Stability', 'Labor Strikes'): 2, ('Political Stability', 'Political Stability'): 1, ('Political Stability', 'Logistic Index'): 1/3,
    ('Logistic Index', 'Digital Prepardness'): 1/3, ('Logistic Index', 'Natural Disasters'): 5, ('Logistic Index', 'Labor Strikes'): 4, ('Logistic Index', 'Political Stability'): 3, ('Logistic Index', 'Logistic Index'): 1
    }


## Calculate the weight for the criteria
The above table represents an n x n comparison matrix which contains the intensities defined by us (Karmaker, C.L et.al, 2018). $$
 M = (w_{i}/w_{j})_{n x n} = (\begin{array}{cc}
w_{1}/w_{1} & w_{1}/w_{2} & .... & w_{1}/w_{n}\\
w_{2}/w_{1} & w_{2}/w_{2} & .... & w_{2}/w_{n}\\
. & . & ..... & . \\
w_{n}/w_{1} & w_{n}/w_{2} & .... & w_{n}/w_{n}
\end{array})
$$

## Generate the weights
The n x n matrix is first normalized using the formula below and a priority vector is generated with which the weights are generated.
$$
W = \begin{bmatrix}
           w_{1} \\
           w_{2} \\
           \vdots \\
           w_{n}
         \end{bmatrix}
$$

In [39]:
supply_chain = ahpy.Compare(name='Supply_Chain', comparisons=supply_chain_comp, precision=3, random_index='saaty')

print(supply_chain.target_weights)
print('Consistency Ratio: ' + str(supply_chain.consistency_ratio))

{'Digital Prepardness': 0.498, 'Logistic Index': 0.26, 'Political Stability': 0.111, 'Natural Disasters': 0.08, 'Labor Strikes': 0.05}
Consistency Ratio: 0.051


## Calculate and check the Consistency ratio
It is assumed that the experts using the AHP makers are objective. However, this is not true in real life which generates a certain level of uncertainty towards this evaluation. Therefore, Saaty solved this issue by creating the consistency Index and consistency ratios. To accept the matrix, the CR value should be less the 0.1 meaning there could be an inconsistency error of 10%.

In [40]:
# All the packages that we need to import
import numpy as np               # for linear algebra
import pandas as pd              # for tabular output
from scipy.stats import rankdata # for ranking the candidates

## Pre-requisites

For this problem, we are always provided with the following data:
1. The ratings in every category for each candidate.
2. The weights for every category or attribute to be considered.

Note that an attribute can be beneficial attribute (in which case, we will want to maximize it's contribution) or a cost attribute (which we will need to minimize). We call the set of beneficial attributes $J_1$ and that of cost attributes $J_2 = J_1^C$.

In [49]:
# The given data encoded into vectors and matrices

attributes = np.array(["Digital Prepardness", "Natural Disasters", "Labor Strikes", "Political Stability", "Logistic Index"])
candidates = np.array(["Germany", "Poland", "France", "Belgium","UK", "Portugal", "Bulgaria", "Netherlands", "Spain", "Ireland", "Hungary"])
raw_data = np.array([
    [88.07, 3.26, 17.30, 0.76, 4.10], # Germany
    [84.09, 3.65, 16.0, 0.51, 3.6], # Poland
    [86.41, 3.53, 127.6, 0.37, 3.9], # France
    [90.69, 3.52, 97.9, 0.61, 4.0], # Belgium
    [85.59, 4.058, 17.9, 0.54, 3.8], # UK
    [85.5, 3.9, 14.1, 0., 3.4], # Portugal
    [64.37, 3.88, 2.1, 0.46, 3.2], # Bulgaria
    [84.66, 7.44, 19.2, 0.92, 4.1], # Netherlands
    [88.61, 2.94, 49.1, 0.58, 3.9], # Spain
    [78.56, 4.2, 15.6, 0.86, 3.6], # Ireland
    [76.74, 4.68, 6.2, 0.86, 3.2], # Hungary
])

weights = np.array([0.498, 0.08, 0.05, 0.111, 0.26])

# The indices of the attributes (zero-based) that are considered beneficial.
# Those indices not mentioned are assumed to be cost attributes.
# Cost benefit functions = when cost (lower is better) and 1 when benefit function (more is better)
# This attribute sets if the colum is better with lower or higher data.
benefit_attributes = set([0, 3, 4])

# Display the raw data we have
pd.DataFrame(data=raw_data, index=candidates, columns=attributes)

,Digital Prepardness,Natural Disasters,Labor Strikes,Political Stability,Logistic Index
Germany,88.07,3.260,17.3,0.76,4.1
Poland,84.09,3.650,16.0,0.51,3.6
France,86.41,3.530,127.6,0.37,3.9
Belgium,90.69,3.520,97.9,0.61,4.0
UK,85.59,4.058,17.9,0.54,3.8
Portugal,85.50,3.900,14.1,0.00,3.4
Bulgaria,64.37,3.880,2.1,0.46,3.2
Netherlands,84.66,7.440,19.2,0.92,4.1
Spain,88.61,2.940,49.1,0.58,3.9
Ireland,78.56,4.200,15.6,0.86,3.6


## Step 1 - Normalizing the ratings

$$r_{ij}=\frac{x_{ij}}{\sqrt{\sum_{i = 1}^{m} x_{ij}^2}}$$

where $i = 1, 2, \ldots, m$ and $j = 1, 2, \ldots, n$.

In [42]:
m = len(raw_data)
n = len(attributes)
divisors = np.empty(n)
for j in range(n):
    column = raw_data[:,j]
    divisors[j] = np.sqrt(column @ column)

raw_data /= divisors

columns = ["$X_{%d}$" % j for j in range(n)]
pd.DataFrame(data=raw_data, index=candidates, columns=columns)

,$X_{0}$,$X_{1}$,$X_{2}$,$X_{3}$,$X_{4}$
Germany,0.318672,0.231053,0.099871,0.357795,0.332074
Poland,0.304271,0.258694,0.092366,0.240099,0.291577
France,0.312666,0.250189,0.736621,0.174190,0.315875
Belgium,0.328152,0.249481,0.565166,0.287177,0.323974
UK,0.309698,0.287611,0.103335,0.254223,0.307776
Portugal,0.309373,0.276413,0.081398,0.000000,0.275378
Bulgaria,0.232916,0.274996,0.012123,0.216560,0.259179
Netherlands,0.306333,0.527311,0.110840,0.433120,0.332074
Spain,0.320626,0.208373,0.283449,0.273054,0.315875
Ireland,0.284261,0.297676,0.090057,0.404873,0.291577


## Step 2 - Calculating the Weighted Normalized Ratings

$$v_{ij} = w_j r_{ij}$$

where $i = 1, 2, \ldots, m$ and $j = 1, 2, \ldots, n$.

In [43]:
raw_data *= weights
pd.DataFrame(data=raw_data, index=candidates, columns=columns)

,$X_{0}$,$X_{1}$,$X_{2}$,$X_{3}$,$X_{4}$
Germany,0.158699,0.018484,0.004994,0.039715,0.086339
Poland,0.151527,0.020696,0.004618,0.026651,0.075810
France,0.155707,0.020015,0.036831,0.019335,0.082127
Belgium,0.163420,0.019958,0.028258,0.031877,0.084233
UK,0.154230,0.023009,0.005167,0.028219,0.080022
Portugal,0.154068,0.022113,0.004070,0.000000,0.071598
Bulgaria,0.115992,0.022000,0.000606,0.024038,0.067387
Netherlands,0.152554,0.042185,0.005542,0.048076,0.086339
Spain,0.159672,0.016670,0.014172,0.030309,0.082127
Ireland,0.141562,0.023814,0.004503,0.044941,0.075810


## Step 3 - Identifying PIS ($A^*$) and NIS ($A^-$)

$$
\begin{align}
A^* &= \left\{v_1^*, v_2^*, \ldots, v_n^*\right\} \\
A^- &= \left\{v_1^-, v_2^-, \ldots, v_n^-\right\} \\
\end{align}
$$

And we define

$$
\begin{align}
v_j^* &=
\begin{cases}
\max{(v_{ij})}, \text{ if} j \in J_1 \\
\min{(v_{ij})}, \text{ if} j \in J_2
\end{cases}
\\
v_j^- &=
\begin{cases}
\min{(v_{ij})}, \text{ if} j \in J_1 \\
\max{(v_{ij})}, \text{ if} j \in J_2
\end{cases}
\\
\end{align}
$$

where $i = 1, 2, \ldots, m$ and $j = 1, 2, \ldots, n$.

In [44]:
a_pos = np.zeros(n)
a_neg = np.zeros(n)
for j in range(n):
    column = raw_data[:,j]
    max_val = np.max(column)
    min_val = np.min(column)
    
    # See if we want to maximize benefit or minimize cost (for PIS)
    if j in benefit_attributes:
        a_pos[j] = max_val
        a_neg[j] = min_val
    else:
        a_pos[j] = min_val
        a_neg[j] = max_val

pd.DataFrame(data=[a_pos, a_neg], index=["$A^*$", "$A^-$"], columns=columns)

,$X_{0}$,$X_{1}$,$X_{2}$,$X_{3}$,$X_{4}$
$A^*$,0.163420,0.016670,0.000606,0.048076,0.086339
$A^-$,0.115992,0.042185,0.036831,0.000000,0.067387


## Step 4 and 5 - Calculating Separation Measures and Similarities to PIS

The separation or distance between the alternatives can be measured by the $n$-dimensional Euclidean distance. The separation from the PIS $A^*$ and NIS $A^-$ are $S^*$ and $S^-$ respectively.

$$
\begin{align}
S_i^* &= \sqrt{\sum_{j = 1}^n \left(v_{ij} - v^*_j\right)^2} \\
S_i^- &= \sqrt{\sum_{j = 1}^n \left(v_{ij} - v^-_j\right)^2} \\
\end{align}
$$

where $i = 1, 2, \ldots, m$ and $j = 1, 2, \ldots, n$.

We also calculate

$$
C^*_i = \frac{S_i^-}{S_i^* + S_i^-},\text{ where }i = 1, 2, \ldots, m
$$

In [45]:
sp = np.zeros(m)
sn = np.zeros(m)
cs = np.zeros(m)

for i in range(m):
    diff_pos = raw_data[i] - a_pos
    diff_neg = raw_data[i] - a_neg
    sp[i] = np.sqrt(diff_pos @ diff_pos)
    sn[i] = np.sqrt(diff_neg @ diff_neg)
    cs[i] = sn[i] / (sp[i] + sn[i])

pd.DataFrame(data=zip(sp, sn, cs), index=candidates, columns=["$S^*$", "$S^-$", "$C^*$"])

,$S^*$,$S^-$,$C^*$
Germany,0.010712,0.073046,0.872112
Poland,0.027270,0.059526,0.685816
France,0.047188,0.051575,0.522207
Belgium,0.032285,0.064162,0.665258
UK,0.024076,0.061550,0.718820
Portugal,0.051553,0.054255,0.512770
Bulgaria,0.056699,0.047932,0.458106
Netherlands,0.028168,0.070614,0.714845
Spain,0.023054,0.064871,0.737796
Ireland,0.025781,0.064242,0.713615


## Step 6 - Ranking the candidates/alternatives

We choose the candidate with the maximum $C^*$ or rank all the alternatives in descending order according to their $C^*$ values. This process can also be done for the $S^*$ and $S^-$ values.

In [46]:
def rank_according_to(data):
    ranks = rankdata(data).astype(int)
    ranks -= 1
    return candidates[ranks][::-1]

In [47]:
cs_order = rank_according_to(cs)
sp_order = rank_according_to(sp)
sn_order = rank_according_to(sn)

pd.DataFrame(data=zip(cs_order, sp_order, sn_order), index=range(1, m + 1), columns=["$C^*$", "$S^*$", "$S^-$"])

,$C^*$,$S^*$,$S^-$
1,Belgium,Netherlands,Portugal
2,Bulgaria,Belgium,Netherlands
3,Ireland,Poland,Spain
4,Netherlands,Portugal,Ireland
5,Germany,Hungary,Germany
6,Poland,Ireland,France
7,Spain,France,UK
8,UK,Bulgaria,Bulgaria
9,France,Spain,Poland
10,Portugal,UK,Belgium


In [48]:
print("The best candidate/alternative according to C* is " + cs_order[0])
print("The preferences in descending order are " + ", ".join(cs_order) + ".")

The best candidate/alternative according to C* is Belgium
The preferences in descending order are Belgium, Bulgaria, Ireland, Netherlands, Germany, Poland, Spain, UK, France, Portugal, Hungary.


## References
Evelyn, E. and EdmondYeboah, N., (2015). Ranking Agricultural Supply Chain Risk in Ghana: An AHP Approach. International Journal of Economics,Commerece and Management, III, 2, pp.1-12.

Hungrybluedev (2021) Topsis implementation, Kaggle. Available from: https://www.kaggle.com/code/hungrybluedev/topsis-implementation/notebook (Accessed: 12 May 2023).

Karmaker, C.L., Ahmed, S.M.T., Rahman, M.S., Tahiduzzaman, M., Biswas, T.K., Rahman, M. and Biswas, S.K., 2018. A framework of faculty performance evaluation: A case study in Bangladesh. International Journal of Research in Advanced Engineering and Technology, 4(3), pp.18-24.

Longaray, A.A., Gois, J.D.D.R. and da Silva Munhoz, P.R., ()2015. Proposal for using AHP method to evaluate the quality of services provided by outsourced companies. Procedia Computer Science, 55, pp.715-724.